In [1]:
import os,sys,inspect
from IPython.display import display, Markdown, Latex, Javascript, clear_output
import emoji
import time
import ipywidgets as widgets
import ipython_blocking
import numpy as np
import time
from tqdm import tqdm_notebook
import functools
import configparser
import webbrowser
import shutil


parentdir = os.path.abspath('../../../pymodules')

sys.path.insert(0,parentdir)
#from connect_device import connect
from connect_device_v2.vizualizate import vizulizated_finds_devices, advanced_search_device,DeviceBundle
from connect_device_v2.save_file import SaveFilesButton
from connect_device_v2.connect import connect_to_device
from connect_device_v2.find_device import FindDevice
from connect_device_v2.auxiliary_commands import check_error,disable_all_correctors,enable_ifbf_for_all_segment

import pandas as pd
from datetime import datetime
button = widgets.Button(
                description='Continue',
                disabled=False,
                button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
                tooltip='Click me',
                icon='play'        
                )
testname="СКО трасс"
display(Markdown('### 💠 {}'.format(testname)))



### 💠 СКО трасс

In [2]:
from connect_device_v2.find_device import FindDevice,parse_config
box = FindDevice()
display(box)

Выбранная конфигурация
filter_if:3 k,300 k
athenuator:10,30,50


<IPython.core.display.Javascript object>

In [11]:
button = box.select_device_continue_button

In [12]:
%blockrun button

In [13]:
button1 = box.select_config_continue_button

In [14]:
%blockrun button1

In [15]:
inst = box.selected_device

In [16]:
date=datetime.now()
date=date.strftime('%Y-%m-%d %H:%M:%S')
device = inst.query("*IDN?")
device = device.replace(' ','')
device = device.split(',')
device_info = {'Время запуска':date,
              'device':device[0],
              'model':device[1],
              'serial':device[2],
              'version':device[3]}
portCount=int(inst.query("SERV:PORT:COUN?"))
traces=["T","R","S"]
trace_count = portCount* len(traces)

In [17]:
result_config =  box.config

In [18]:
# def check_error():
#     responce = inst.query('SYST:ERR?')    
#     if "No error" not in responce:        
#         print(responce)
#         #pass
#     else:        
#         pass
#         #print('succes')

In [19]:
def parse_config(config: dict):
    factor_value = {
        'k': 10 ** 3,
        'm': 10 ** 6,
        'g': 10 ** 9
    }
    new_config = config.copy()
    
    if_value = config['DEVICE']['filter_if'].split(',') 
    
    new_if_value=''
    for i in if_value:
        numeric, factor = i.split(' ')
        new_if_value += f'{float(numeric) * factor_value[factor.lower()]} ' 
    new_config['DEVICE']['filter_if']=new_if_value
   
    return new_config

In [20]:
new_config=parse_config(dict(result_config))



In [21]:
if_filter_value=new_config['DEVICE']['filter_if'].split(' ')[:-1:]
athenuator_value = new_config['DEVICE']['athenuator'].split(',')

In [22]:
# def enable_ifbf_for_all_segment(inst):
#     data = inst.query(f'SENSe1:SEGMent:DATA?').split(',')
#     data[2]=0
#     str1 = ''.join(str(e)+',' for e in data)
#     inst.write(f'SENSe1:SEGMent:DATA {str1[:-1]}')
#     #check_error(inst)

In [23]:
#Сброс по умолчанию и остановка сканирования
#inst.write("*RST")
inst.write("SYST:PRES")

#отключить все корректоры
disable_all_correctors(inst)
inst.write("SERVice:SWEep:FREQuency:FACTory")
check_error(inst)
enable_ifbf_for_all_segment(inst)
# прерводим прибор в режим внешнего управления и останавливаем развёртку
inst.write("TRIG:SOUR BUS")
check_error(inst)
inst.write("INIT:CONT OFF")
check_error(inst)
#inst.write("DISP:WIND:TRAC:Y:PDIV 0.01")
check_error(inst)

# атенюатор SERVice:RFCTL:POWer:STATe 1
inst.write("SERVice:RFCTL:POWer:STATe 1")
inst.write(f"SERVice:RFCTL:POWer:ATT 10")
inst.write(f"SERV:RFCTL:POW:DAC 6554")
check_error(inst)
inst.write('CALC:MST 1')
######################################################

-215, Invalid segment data


(11, <StatusCode.success: 0>)

In [24]:
def finc(traceCategory,trace,filtr,att,port):
    inst.write(f"SENS:BAND {filtr}")
    check_error(inst)
    
    if (traceCategory != 'S'):        
        #print(f"CALC:PAR1:DEF {trace+str(port)}")
        inst.write(f"CALC:PAR1:DEF {trace+str(port)}")
        check_error(inst)
        
        #print(f"CALC:PAR1:SPOR {port}")
        inst.write(f"CALC:PAR1:SPOR {port}")
        check_error(inst)
    else:
        #print(f"CALC:PAR1:DEF {trace+str(port)*2}")
        inst.write(f"CALC:PAR1:DEF {trace+str(port)*2}")
        check_error(inst)

    #print(f"CALC:PAR1:SPOR {port}")
    inst.write(f"CALC:PAR1:SPOR {port}")
    check_error(inst)

    #print("SERVice:RFCTL:POWer:STATe 1")
    inst.write("SERVice:RFCTL:POWer:STATe 1")
    check_error(inst)

    #print(f"SERVice:RFCTL:POWer:ATT {att}")
    inst.write(f"SERVice:RFCTL:POWer:ATT {att}")
    check_error(inst)

    start_scan()
    
    normalizeited_graf(1)
    #сделать измерения
    msd = get_msd()
    traceName =inst.query("CALC:PAR1:DEF?")
    check_error(inst)
    att_devices=inst.query_ascii_values("SERVice:RFCTL:POWer:ATT?")
    check_error(inst)                            
    filtr_devices=inst.query_ascii_values("SENS:BAND?")
    check_error(inst)
    return traceName,filtr_devices,att_devices,msd

In [25]:
# def go_msd(table):
#     for trace in traces:
#             for port in range(1,portCount+1):
#                 for att in athenuator_value:
#                     for filtr in if_filter_value:
#                         if (trace!="S"):
#                             traceCategory="TR"
#                             print(f"SENS:BAND {filtr}")
#                             inst.write(f"SENS:BAND {filtr}")
#                             check_error()
                            
#                             print(f"CALC:PAR1:DEF {trace+str(port)}")
#                             inst.write(f"CALC:PAR1:DEF {trace+str(port)}")
#                             check_error()
                            
#                             print(f"CALC:PAR1:SPOR {port}")
#                             inst.write(f"CALC:PAR1:SPOR {port}")
#                             check_error()
                            
#                             print("SERVice:RFCTL:POWer:STATe 1")
#                             inst.write("SERVice:RFCTL:POWer:STATe 1")
#                             check_error()
                            
#                             print(f"SERVice:RFCTL:POWer:ATT {att}")
#                             inst.write(f"SERVice:RFCTL:POWer:ATT {att}")
#                             check_error()
                            
#                             start_scan()
#                             #time.sleep(1)
#                             normalizeited_graf(1)
#                             #сделать измерения
#                             msd = get_msd()
#                             traceName =inst.query("CALC:PAR1:DEF?")
#                             check_error()
#                             att_devices=inst.query_ascii_values("SERVice:RFCTL:POWer:ATT?")
#                             check_error()                            
#                             filtr_devices=inst.query_ascii_values("SENS:BAND?")
#                             check_error()
#                         else:
#                             traceCategory="S"
#                             inst.write(f"SENS:BAND {filtr}")
#                             check_error()
#                             inst.write(f"CALC:PAR1:DEF {trace+str(port)*2}")
#                             check_error()
#                             #inst.write(f"CALC:PAR1:SPOR {port}")
#                             inst.write("SERVice:RFCTL:POWer:STATe 1")
#                             check_error()
#                             inst.write(f"SERVice:RFCTL:POWer:ATT {att}")
#                             check_error()
#                             start_scan()
#                             normalizeited_graf(1)
#                             #сделать измерения
#                             msd = get_msd()
#                             traceName =inst.query("CALC:PAR1:DEF?")
#                             att_devices=inst.query_ascii_values("SERVice:RFCTL:POWer:ATT?")
#                             filtr_devices=inst.query_ascii_values("SENS:BAND?")
#                         table.append((traceCategory,traceName,filtr_devices,att_devices,msd))
#                         pbar.update()
#     pbar.close()
#     return(table)
#                         #print(trace,port,att,filtr,msd)
#                         #time.sleep()



In [26]:
def go_msd(table):
    for trace in traces:
            for port in range(1,portCount+1):
                for att in athenuator_value:
                    for filtr in if_filter_value:
                        if (trace!="S"):
                            traceCategory="TR"
                            traceName,filtr_devices,att_devices,msd = finc(traceCategory,
                                                                           trace,
                                                                           filtr,
                                                                           att,
                                                                           port)
                        else:
                            traceCategory="S"
                            traceName,filtr_devices,att_devices,msd = finc(traceCategory,
                                                                           trace,
                                                                           filtr,
                                                                           att,
                                                                           port)
                        table.append((traceCategory,traceName,filtr_devices,att_devices,msd))
                        pbar.update()
    pbar.close()
    return(table)
                        #print(trace,port,att,filtr,msd)
                        #time.sleep()

In [27]:
def get_msd():
    start_scan()        
    a=float(inst.query_ascii_values("CALC:MST:DATA?")[1])    
    return(a)

In [28]:
def start_scan():    
    inst.write("INIT")
    check_error(inst)
    inst.write("TRIG:SING")
    # Читаем ошибки и ждём окончания для считывания данных, затем всё то же самое для порта 2
    #print('OPC',inst.query("*OPC?"))   
    inst.query("*OPC?")
    check_error(inst)

In [29]:
def normalizeited_graf(n_graf):
    inst.write(f"CALC:PAR{n_graf}:SEL")
    check_error(inst)
    inst.write("CALC:MATH:MEM")
    check_error(inst)
    inst.write("CALC:MATH:FUNC DIVide")
    check_error(inst)
    #inst.write(f"DISP:WIND:TRAC{n_graf}:MEM 0")
    #check_error()

In [30]:

inst.timeout=30000
pbar = tqdm_notebook(total=trace_count*len(athenuator_value)*len(if_filter_value))
table=[]
result= go_msd(table)


In [31]:
def table_generator(table):
    columns=["Канал","ПЧ, Гц","Аттенюатор, дБ","Измерение, дБ","Максимум, дБ","Допуск, дБ"]
    html_table='<table border="1" ><tr>'
    for i in columns:        
        html_table+=f"<td>{i}</td>"
    html_table+="</tr>"
    counter=0
    for i in table:        
        html_table+=f"<tr><td>{i[1]}</td>"
        #print("i=",i[0], 'counter=', counter)
        if (i[0] == "TR" and counter == 0):
            #print("Зашли в tr")
            index=table.index(i)
            msd=max_msd(table,index,index+(portCount*2))
            td_background=""
            if(msd>addict):
                 td_background="bgcolor=\"#ffdd00\""
            html_table += f"<td rowspan={portCount*2}>{i[2]}</td> \
                          <td rowspan={portCount*2}>{i[3]}</td> \
                          <td>{i[4]}</td> \
                          <td rowspan={portCount*2} {td_background}>{msd}</td> \
                          <td rowspan={portCount*2} contenteditable='true' {td_background}>{addict}</td>"
            counter += 1
            continue
        if (i[0] == "S" and counter == portCount*2):
            #print("Зашли в s")
            index=table.index(i)
            msd=max_msd(table,index,index+(portCount))
            if(msd>addict):
                 td_background="bgcolor=\"#ffdd00\""
            html_table += f"<td rowspan={portCount}>{i[2]}</td> \
                          <td rowspan={portCount}>{i[3]}</td> \
                          <td >{i[4]}</td> \
                          <td rowspan={portCount} {td_background}>{msd}</td> \
                          <td rowspan={portCount} contenteditable='true' {td_background}>{addict}</td>"
            counter +=1
            
            continue
        else:
            #print(" Зашли в qwe")
            html_table += f"<td >{i[4]}</td>"                          
            counter+=1
            if(i[0]=="S" and counter == portCount*len(traces)): counter=0
            continue
        html_table += "</tr>"       

    
    html_table += "</table>"
    
    return(html_table)

In [32]:
def max_msd(table,index_start,index_stop):
    arr=[]
    for i in table[index_start:index_stop]:
        arr.append(i[4])
    return(max(arr))

In [ ]:
mark=Markdown('### 💠 Укажите допуск Дб')
text=widgets.Text(
        value='',
        placeholder='Допуск',
        description='Допуск',
        disabled=False)
button = widgets.Button(
    description='Continue',
    disabled=False,
    button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='play'        
    )
display(mark,text,button)

In [ ]:
%blockrun button

In [ ]:
from IPython.core.display import display, HTML
addict=float(text.value.replace(',','.'))
html_data = table_generator((sorted(result, key=lambda point: (point[2], point[3]))))
display(HTML(html_data))

In [ ]:
file_data = html_data
my_button = SaveFilesButton(file_data)
my_button # This will display the button in the context of Jupyter Notebook